#### Load Libraries

In [ ]:
import psycopg2
from os import getenv
from dotenv import load_dotenv
from urllib.parse import quote
import pandas as pd
from IPython.display import display, HTML
from sqlalchemy import create_engine
import io
import urllib

load_dotenv(override=True)
export_prompt = 'Enter file name for the export:'
DM_USER = getenv('DM_USER')
DM_PASS = quote(getenv('DM_PASS'))
DB_CONFIG = {
    "DB_NAME": getenv("DB_NAME"),
    "DB_USER": getenv("DB_USER"),
    "DB_PASS": getenv("DB_PASS"),
    "DB_HOST": getenv("DB_HOST"),
    "DB_PORT": getenv("DB_PORT"),
}
conn_str = f"postgresql+psycopg2://{DB_CONFIG['DB_USER']}:{DB_CONFIG['DB_PASS']}@{DB_CONFIG['DB_HOST']}:{DB_CONFIG['DB_PORT']}/{DB_CONFIG['DB_NAME']}"
csv_buffer = io.StringIO()
download_btn_template = '''
<a href="data:text/csv;charset=utf-8,{csv_content}" download="{file_name}.csv" id="downloadBtn">Download CSV</a>

<style>
    #downloadBtn {{
        padding: 12px 24px;
        font-size: 18px;
        border-radius: 8px;
        background-color: #007BFF;
        color: white;
        border: none;
        cursor: pointer;
        margin-top: 10px;
        text-decoration: none;
        display: inline-block;
        margin-left: 20px; 
        transition: transform 0.3s ease;
    }}
    
    #downloadBtn:hover {{
        transform: scale(1.08);
    }}
    
    #downloadBtn:active {{
        transform: scale(1);
        animation: flash 0.5s;
    }}
    
    @keyframes flash {{
        0% {{ background-color: #007BFF; }}
        50% {{ background-color: #FF8C00; }}
        100% {{ background-color: #007BFF; }}
    }}
</style>

<div style="margin-bottom: 20px;"></div>
'''

#### View Writeup Data

In [ ]:
def fetch_data_from_db(query, conn_str):
    engine = create_engine(conn_str)
    df = pd.read_sql(query, engine)
    return df
    
df_writeup = fetch_data_from_db("SELECT * FROM ELN_WRITEUP_API_EXTRACT", conn_str)
display(HTML(df_writeup.to_html()))

### View Similarity Results

In [ ]:
df_compr = fetch_data_from_db("SELECT * FROM ELN_WRITEUP_COMPARISON", conn_str)
display(HTML(df_compr.to_html()))

#### Export Dataframe as html

In [ ]:
# df_name = input('select the data to export (drop-down list would be good here)')
file_name = input(export_prompt)
data_df.to_csv(csv_buffer, index=False)
csv_content = csv_buffer.getvalue()
csv_content = urllib.parse.quote(csv_content)
download_link = download_btn_template.format(csv_content=csv_content, file_name=file_name)
display(HTML(download_link))